In [1]:
import pandas as pd
import os, glob

folder_path = '/app/data/test_set/GOOG'

message_files = glob.glob(os.path.join(folder_path, '*message*'))
results = []

for file_path in message_files:
    filename = os.path.basename(file_path)

    # Skip .npy files
    if filename.endswith(".npy"):
        continue

    try:
        df = pd.read_csv(file_path, header=None, low_memory=False,
                         names=['Time','Event_Type','Order_ID','Size','Price','Direction','Extra'])

        valid_prices = df[(df['Price'] >= 500000) & (df['Price'] <= 20000000)]['Price']

        if len(valid_prices) > 0:
            highest_price = valid_prices.max()
            lowest_price = valid_prices.min()
        else:
            highest_price = float('nan')
            lowest_price = float('nan')

        all_message_ids = df['Order_ID'].unique().astype(int).tolist()
        execution_sum = df.loc[df['Event_Type'] == 4, 'Size'].sum()

        results.append({
            'filename': filename,
            'highest_price': highest_price,
            'lowest_price': lowest_price,
            'all_message_ids': all_message_ids,
            'execution_sum': execution_sum
        })

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        results.append({
            'filename': filename,
            'highest_price': float('nan'),
            'lowest_price': float('nan'),
            'all_message_ids': [],
            'execution_sum': 0
        })

daily_h_l_df = pd.DataFrame(results)

print(f"Processed {len(results)} files")
print(daily_h_l_df)

Processed 9 files
                                           filename  highest_price  \
0  GOOG_2023-01-03_34200000_57600000_message_10.csv         916400   
1  GOOG_2023-01-13_34200000_57600000_message_10.csv         930700   
2  GOOG_2023-01-04_34200000_57600000_message_10.csv         915100   
3  GOOG_2023-01-05_34200000_57600000_message_10.csv         883700   
4  GOOG_2023-01-09_34200000_57600000_message_10.csv         909200   
5  GOOG_2023-01-11_34200000_57600000_message_10.csv         924400   
6  GOOG_2023-01-10_34200000_57600000_message_10.csv         895700   
7  GOOG_2023-01-06_34200000_57600000_message_10.csv         885700   
8  GOOG_2023-01-12_34200000_57600000_message_10.csv         928100   

   lowest_price                                    all_message_ids  \
0        889200  [26617106, 26628478, 26628606, 26628742, 26629...   
1        908400  [27238094, 27238534, 27242942, 27242954, 27245...   
2        877100  [27699218, 27719870, 27720354, 27720530, 27721...   
3

In [2]:
# Check for repetitions in all_message_ids between rows
print("Checking for repetitions in all_message_ids between rows...")

# Get all message IDs from all rows
all_ids_combined = []
for idx, row in daily_h_l_df.iterrows():
    all_ids_combined.extend(row['all_message_ids'])

# Check if there are any duplicates
unique_ids = set(all_ids_combined)
total_ids = len(all_ids_combined)
unique_count = len(unique_ids)

print(f"Total message IDs across all files: {total_ids}")
print(f"Unique message IDs: {unique_count}")
print(f"Duplicates found: {total_ids - unique_count}")

if total_ids != unique_count:
    print("\nWARNING: There are repeated message IDs between files!")
    
    # Find which IDs are duplicated
    from collections import Counter
    id_counts = Counter(all_ids_combined)
    duplicated_ids = {id_val: count for id_val, count in id_counts.items() if count > 1}
    
    print(f"Number of duplicated IDs: {len(duplicated_ids)}")
    if len(duplicated_ids) <= 10:  # Show details only if not too many
        print("Duplicated IDs and their counts:")
        for id_val, count in duplicated_ids.items():
            print(f"  ID {id_val}: appears {count} times")
else:
    print("✓ No repetitions found - all message IDs are unique across files")


Checking for repetitions in all_message_ids between rows...
Total message IDs across all files: 10884080
Unique message IDs: 10640827
Duplicates found: 243253

Number of duplicated IDs: 239676


In [3]:
# Save the dataframe to CSV
daily_h_l_df.to_csv('daily_h_l.csv', index=False)
print("Saved daily_h_l_df to daily_h_l.csv")


Saved daily_h_l_df to daily_h_l.csv


# Calculate mean(ln(H/L)) from the daily high/low data

In [4]:
# Calculate mean(ln(H/L)) from the daily high/low data
import numpy as np

# Filter out rows with NaN values
valid_data = daily_h_l_df.dropna(subset=['highest_price', 'lowest_price'])

# Calculate ln(H/L) for each valid row
# Note: prices are in LOBSTER format (multiplied by 10000), but the ratio H/L cancels out the scaling
ln_h_l_ratios = np.log(valid_data['highest_price'] / valid_data['lowest_price'])

# Calculate the mean
mean_ln_h_l = ln_h_l_ratios.mean()

print(f"Number of valid files for calculation: {len(valid_data)}")
print(f"Mean(ln(H/L)): {mean_ln_h_l:.6f}")

# Display some statistics
print(f"\nStatistics for ln(H/L):")
print(f"Min: {ln_h_l_ratios.min():.6f}")
print(f"Max: {ln_h_l_ratios.max():.6f}")
print(f"Std: {ln_h_l_ratios.std():.6f}")


Number of valid files for calculation: 9
Mean(ln(H/L)): 0.030206

Statistics for ln(H/L):
Min: 0.021735
Max: 0.042412
Std: 0.006498
